In [1]:
import gradio as gr
import json
import pickle

/home/betty/miniconda3/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
with open('dialogues.pkl', 'rb') as f:
    questions = pickle.load(f)

In [3]:
# 检查加载的数据
questions[0]
# questions[0]['metadata']
# questions[0]['script']
# questions[0]['conversation']['conversation'][0]

{'metadata': {'setting': {'location': 'ElectroHub',
   'time_of_day': 'afternoon',
   'context': 'busy shopping period'},
  'role_1': {'name': 'Alexandra Thompson',
   'gender': 'female',
   'age': 28,
   'occupation': 'sales associate',
   'nationality': 'Canadian',
   'personality_traits': ['patient', 'knowledgeable', 'enthusiastic'],
   'relationship_context': 'store employee assisting customer',
   'self_introduction': "Alexandra, or Alex, has been working at ElectroHub for three years. She's known for her extensive knowledge of the latest gadgets and her ability to explain complex technology in simple terms. Her patience and warm smile make even the most tech-phobic customers feel at ease. Alex is particularly enthusiastic about smart home devices and is always eager to share her insights on how they can make life easier and more convenient."},
  'role_2': {'name': 'Ethan Patel',
   'gender': 'male',
   'age': 35,
   'occupation': 'marketing manager',
   'nationality': 'American',

In [2]:
# # Metadata definition
# metadata = {
#     'setting': {
#         'location': 'ElectroHub',
#         'time_of_day': 'afternoon',
#         'context': 'busy shopping period'
#     },
#     'role_1': {
#         'name': 'Alexandra Thompson',
#         'gender': 'female',
#         'age': 28,
#         'occupation': 'sales associate',
#         'nationality': 'Canadian',
#         'personality_traits': ['patient', 'knowledgeable', 'enthusiastic'],
#         'relationship_context': 'store employee assisting customer',
#         'self_introduction': "Alexandra, or Alex, has been working at ElectroHub for three years. She's known for her extensive knowledge of the latest gadgets and her ability to explain complex technology in simple terms. Her patience and warm smile make even the most tech-phobic customers feel at ease. Alex is particularly enthusiastic about smart home devices and is always eager to share her insights on how they can make life easier and more convenient."
#     },
#     'role_2': {
#         'name': 'Ethan Patel',
#         'gender': 'male',
#         'age': 35,
#         'occupation': 'marketing manager',
#         'nationality': 'American',
#         'personality_traits': ['inquisitive', 'tech-savvy', 'somewhat skeptical'],
#         'relationship_context': 'customer seeking advice on a product',
#         'self_introduction': "Ethan is a busy marketing manager who's always on the lookout for the latest technology to enhance his work and personal life. He's particularly interested in smart home security systems and is considering upgrading his current setup. Ethan is inquisitive and likes to do his research before making a purchase, often reading reviews and comparing features. Despite his tech-savviness, he can be somewhat skeptical of new products and needs convincing demonstrations of their benefits."
#     },
#     'conversation_context': {
#         'type': 'product inquiry',
#         'main_topic': 'smart home security systems',
#         'relationship_dynamic': 'customer-sales associate interaction',
#         'emotional_tone': 'informative with a hint of persuasion',
#         'expected_duration': '10 minutes',
#         'expected_turns': 10,
#         'key_points': [
#             'Discussion of current smart home security systems available',
#             "Ethan's specific requirements and concerns",
#             "Alex's demonstration of a recommended system",
#             'Comparison of different system features and pricing',
#             "Ethan's questions about system integration and compatibility",
#             "Alex's assurance of after-sales support and warranty"
#         ]
#     },
#     'input_scenario': {
#         'interaction_type': 'retail_service',
#         'location': 'retail_store',
#         'goal': 'product_inquiry',
#         'cultural_background': 'western'
#     }
# }

# # Function to render metadata

# def render_metadata():
#     with gr.Blocks() as demo:
#         with gr.Row():
#             gr.Markdown("### Setting")
#         with gr.Row():
#             gr.Textbox(value=metadata['setting']['location'], label="Location")
#             gr.Textbox(value=metadata['setting']['time_of_day'], label="Time of Day")
#             gr.Textbox(value=metadata['setting']['context'], label="Context")

#         with gr.Row():
#             gr.Markdown("### Role 1")
#         with gr.Row():
#             gr.Textbox(value=metadata['role_1']['name'], label="Name")
#             gr.Textbox(value=metadata['role_1']['gender'], label="Gender")
#         with gr.Row():
#             gr.Textbox(value=str(metadata['role_1']['age']), label="Age")
#             gr.Textbox(value=metadata['role_1']['occupation'], label="Occupation")
#         with gr.Row():
#             gr.Textbox(value=metadata['role_1']['nationality'], label="Nationality")
#             gr.Textbox(value=", ".join(metadata['role_1']['personality_traits']), label="Personality Traits")
#         with gr.Row():
#             gr.Textbox(value=metadata['role_1']['relationship_context'], label="Relationship Context")
#             gr.Textbox(value=metadata['role_1']['self_introduction'], label="Self Introduction")

#         with gr.Row():
#             gr.Markdown("### Role 2")
#         with gr.Row():
#             gr.Textbox(value=metadata['role_2']['name'], label="Name")
#             gr.Textbox(value=metadata['role_2']['gender'], label="Gender")
#         with gr.Row():
#             gr.Textbox(value=str(metadata['role_2']['age']), label="Age")
#             gr.Textbox(value=metadata['role_2']['occupation'], label="Occupation")
#         with gr.Row():
#             gr.Textbox(value=metadata['role_2']['nationality'], label="Nationality")
#             gr.Textbox(value=", ".join(metadata['role_2']['personality_traits']), label="Personality Traits")
#         with gr.Row():
#             gr.Textbox(value=metadata['role_2']['relationship_context'], label="Relationship Context")
#             gr.Textbox(value=metadata['role_2']['self_introduction'], label="Self Introduction")

#         with gr.Row():
#             gr.Markdown("### Conversation Context")
#         with gr.Row():
#             gr.Textbox(value=metadata['conversation_context']['type'], label="Type")
#             gr.Textbox(value=metadata['conversation_context']['main_topic'], label="Main Topic")
#         with gr.Row():
#             gr.Textbox(value=metadata['conversation_context']['relationship_dynamic'], label="Relationship Dynamic")
#             gr.Textbox(value=metadata['conversation_context']['emotional_tone'], label="Emotional Tone")
#         with gr.Row():
#             gr.Textbox(value=metadata['conversation_context']['expected_duration'], label="Expected Duration")
#             gr.Textbox(value=str(metadata['conversation_context']['expected_turns']), label="Expected Turns")
#         with gr.Row():
#             gr.Textbox(value="\n".join(metadata['conversation_context']['key_points']), label="Key Points")

#         with gr.Row():
#             gr.Markdown("### Input Scenario")
#         with gr.Row():
#             gr.Textbox(value=metadata['input_scenario']['interaction_type'], label="Interaction Type")
#             gr.Textbox(value=metadata['input_scenario']['location'], label="Location")
#         with gr.Row():
#             gr.Textbox(value=metadata['input_scenario']['goal'], label="Goal")
#             gr.Textbox(value=metadata['input_scenario']['cultural_background'], label="Cultural Background")

#     return demo

# # Launch the Gradio app
# if __name__ == "__main__":
#     app = render_metadata()
#     app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
current_index = 0

# Load question metadata

def load_metadata(question_index):
    metadata = questions[question_index]["metadata"]
    return metadata

# Gradio Interface for Questions

def interface():
    global current_index

    # Inputs for each metadata field
    interaction_type = gr.Textbox(label="Interaction Type", interactive=False)
    location = gr.Textbox(label="Location", interactive=False)
    goal = gr.Textbox(label="Goal", interactive=False)
    cultural_background = gr.Textbox(label="Cultural Background", interactive=False)

    setting_location = gr.Textbox(label="Setting Location", interactive=False)
    time_of_day = gr.Textbox(label="Time of Day", interactive=False)
    context = gr.Textbox(label="Context", interactive=False)

    role_1_name = gr.Textbox(label="Role 1 Name", interactive=False)
    role_1_gender = gr.Textbox(label="Role 1 Gender", interactive=False)
    role_1_age = gr.Textbox(label="Role 1 Age", interactive=False)
    role_1_occupation = gr.Textbox(label="Role 1 Occupation", interactive=False)
    role_1_nationality = gr.Textbox(label="Role 1 Nationality", interactive=False)
    role_1_personality_traits = gr.Textbox(label="Role 1 Personality Traits", interactive=False)
    role_1_relationship_context = gr.Textbox(label="Role 1 Relationship Context", interactive=False)
    role_1_self_introduction = gr.Textbox(label="Role 1 Self Introduction", interactive=False, lines=5)

    role_2_name = gr.Textbox(label="Role 2 Name", interactive=False)
    role_2_gender = gr.Textbox(label="Role 2 Gender", interactive=False)
    role_2_age = gr.Textbox(label="Role 2 Age", interactive=False)
    role_2_occupation = gr.Textbox(label="Role 2 Occupation", interactive=False)
    role_2_nationality = gr.Textbox(label="Role 2 Nationality", interactive=False)
    role_2_personality_traits = gr.Textbox(label="Role 2 Personality Traits", interactive=False)
    role_2_relationship_context = gr.Textbox(label="Role 2 Relationship Context", interactive=False)
    role_2_self_introduction = gr.Textbox(label="Role 2 Self Introduction", interactive=False, lines=5)

    conversation_type = gr.Textbox(label="Conversation Type", interactive=False)
    main_topic = gr.Textbox(label="Main Topic", interactive=False)
    relationship_dynamic = gr.Textbox(label="Relationship Dynamic", interactive=False)
    emotional_tone = gr.Textbox(label="Emotional Tone", interactive=False)
    expected_duration = gr.Textbox(label="Expected Duration", interactive=False)
    expected_turns = gr.Textbox(label="Expected Turns", interactive=False)
    key_points = gr.Textbox(label="Key Points", interactive=False, lines=5)

    next_button = gr.Button("Next Question")

    # Update Metadata function
    def update_metadata():
        global current_index
        if current_index < len(questions):
            metadata = load_metadata(current_index)
            current_index += 1
            return (
                metadata['input_scenario']['interaction_type'],
                metadata['input_scenario']['location'],
                metadata['input_scenario']['goal'],
                metadata['input_scenario']['cultural_background'],

                metadata['setting']['location'],
                metadata['setting']['time_of_day'],
                metadata['setting']['context'],

                metadata['role_1']['name'],
                metadata['role_1']['gender'],
                str(metadata['role_1']['age']),
                metadata['role_1']['occupation'],
                metadata['role_1']['nationality'],
                ", ".join(metadata['role_1']['personality_traits']),
                metadata['role_1']['relationship_context'],
                metadata['role_1']['self_introduction'],

                metadata['role_2']['name'],
                metadata['role_2']['gender'],
                str(metadata['role_2']['age']),
                metadata['role_2']['occupation'],
                metadata['role_2']['nationality'],
                ", ".join(metadata['role_2']['personality_traits']),
                metadata['role_2']['relationship_context'],
                metadata['role_2']['self_introduction'],

                metadata['conversation_context']['type'],
                metadata['conversation_context']['main_topic'],
                metadata['conversation_context']['relationship_dynamic'],
                metadata['conversation_context']['emotional_tone'],
                metadata['conversation_context']['expected_duration'],
                str(metadata['conversation_context']['expected_turns']),
                "\n".join(metadata['conversation_context']['key_points']),
            )
        else:
            return ("",) * 30

    with gr.Blocks() as app:
        gr.Markdown("# Questionnaire Metadata Viewer")

        with gr.Tab("Input Scenario"):
            with gr.Row():
                interaction_type.render()
                location.render()
            with gr.Row():
                goal.render()
                cultural_background.render()

        with gr.Tab("Setting"):
            with gr.Row():
                setting_location.render()
                time_of_day.render()
            with gr.Row():
                context.render()

        with gr.Tab("Role 1"):
            with gr.Row():
                role_1_name.render()
                role_1_gender.render()
            with gr.Row():
                role_1_age.render()
                role_1_occupation.render()
            with gr.Row():
                role_1_nationality.render()
                role_1_personality_traits.render()
            with gr.Row():
                role_1_relationship_context.render()
            role_1_self_introduction.render()

        with gr.Tab("Role 2"):
            with gr.Row():
                role_2_name.render()
                role_2_gender.render()
            with gr.Row():
                role_2_age.render()
                role_2_occupation.render()
            with gr.Row():
                role_2_nationality.render()
                role_2_personality_traits.render()
            with gr.Row():
                role_2_relationship_context.render()
            role_2_self_introduction.render()

        with gr.Tab("Conversation Context"):
            with gr.Row():
                conversation_type.render()
                main_topic.render()
            with gr.Row():
                relationship_dynamic.render()
                emotional_tone.render()
            with gr.Row():
                expected_duration.render()
                expected_turns.render()
            key_points.render()

        next_button.render()

        # Click Next Button to Update Metadata
        next_button.click(
            fn=update_metadata,
            inputs=[],
            outputs=[
                interaction_type,
                location,
                goal,
                cultural_background,
                
                setting_location,
                time_of_day,
                context,
                
                role_1_name,
                role_1_gender,
                role_1_age,
                role_1_occupation,
                role_1_nationality,
                role_1_personality_traits,
                role_1_relationship_context,
                role_1_self_introduction,

                role_2_name,
                role_2_gender,
                role_2_age,
                role_2_occupation,
                role_2_nationality,
                role_2_personality_traits,
                role_2_relationship_context,
                role_2_self_introduction,

                conversation_type,
                main_topic,
                relationship_dynamic,
                emotional_tone,
                expected_duration,
                expected_turns,
                key_points,
            ]
        )

    return app

# Launch Gradio App
if __name__ == "__main__":
    app = interface()
    app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
